In [6]:
import json
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema


In [9]:
# =========================================
# 1️⃣ LOAD DATA FORMASI
# =========================================
cpns_file = '/Users/muhammadzuamaalamin/Documents/risetmandiir/project/AIRA/helpsekfix/data/data.json'

with open(cpns_file, 'r', encoding='utf-8') as f:
    raw_cpns = json.load(f)

if isinstance(raw_cpns, dict):
    raw_cpns = [raw_cpns]

formasi_docs = []
for idx, item in enumerate(raw_cpns):
    text_parts = [
        "[FORMASI ASN]",
        f"Jabatan: {item.get('jabatan', 'Tidak disebutkan')}",
        f"Instansi: {item.get('instansi', 'Tidak disebutkan')}",
        f"Penempatan: {item.get('penempatan', 'Tidak disebutkan')}",
        f"Jenis Formasi: {item.get('jenis_formasi', 'Tidak disebutkan')}",
        f"Khusus Disabilitas: {'Ya' if item.get('khusus_disabilitas', False) else 'Tidak'}",
        f"Gaji (juta): {item.get('penghasilan', {}).get('min', 0)} - {item.get('penghasilan', {}).get('max', 0)}",
        f"Jumlah Kebutuhan: {item.get('jumlah_kebutuhan', 0)}"
    ]

    pendidikan = item.get('kualifikasi_pendidikan', [])
    if isinstance(pendidikan, str):
        pendidikan = [pendidikan]
    text_parts.append("Kualifikasi Pendidikan: " + "; ".join(pendidikan))

    page_content = "\n".join(text_parts)
    metadata = {
        "tipe": "formasi",
        "instansi": item.get("instansi"),
        "penempatan": item.get("penempatan"),
        "jabatan": item.get("jabatan"),
        "index": idx
    }
    formasi_docs.append(Document(page_content=page_content, metadata=metadata))


# =========================================
# 2️⃣ LOAD DATA FAQ
# =========================================
faq_file = '/Users/muhammadzuamaalamin/Documents/risetmandiir/helpsek/data/Regulasi_Penerimaan_CPNS(AutoRecovered).json'

with open(faq_file, 'r', encoding='utf-8') as f:
    raw_faq = json.load(f)

faq_docs = []
for idx, item in enumerate(raw_faq):
    text = (
        "[FAQ ASN]\n"
        f"Pertanyaan: {item.get('Pertanyaan (FAQ)', 'Tidak disebutkan')}\n"
        f"Jawaban: {item.get('Jawaban', 'Tidak disebutkan')}\n"
        f"Regulasi: {item.get('Regulasi yang Menjadi Dasar', '-')}\n"
        f"Sumber: {item.get('Sumber', '-')}"
    )
    faq_docs.append(Document(page_content=text, metadata={"tipe": "faq", "index": idx}))


In [ ]:
# =========================================
# 3️⃣ BUAT EMBEDDING DAN FAISS INDEX TERPISAH
# =========================================
embeddings = HuggingFaceEmbeddings(
    model_name="/Users/muhammadzuamaalamin/Documents/fintunellm/model/bge-m3",
    model_kwargs={"device": "cpu"}
)
# 1️⃣ Load embedding model dari Hugging Face Hub
# model_name = "intfloat/multilingual-e5-large-instruct"
# embedding_model = HuggingFaceEmbeddings(
#     model_name="BAAI/bge-m3",  # atau "intfloat/multilingual-e5-large-instruct"
#     model_kwargs={'device': 'cuda'},
#     encode_kwargs={'normalize_embeddings': True}
# )


db_formasi = FAISS.from_documents(formasi_docs, embeddings)
db_formasi.save_local("faiss_index_formasi")

db_faq = FAISS.from_documents(faq_docs, embeddings)
db_faq.save_local("faiss_index_faq")

print(f"✅ Total formasi: {len(formasi_docs)}, FAQ: {len(faq_docs)}")


✅ Total formasi: 30, FAQ: 52


In [11]:

# =========================================
# 4️⃣ DETEKSI JENIS PERTANYAAN
# =========================================
def detect_query_type(query: str) -> str:
    query_lower = query.lower()
    formasi_keywords = ["formasi", "jabatan", "penempatan", "instansi", "gaji", "kualifikasi", "unit kerja"]
    faq_keywords = ["apa itu", "bagaimana", "aturan", "dasar hukum", "pppk", "cpns", "asn", "sscasn"]

    if any(kw in query_lower for kw in formasi_keywords):
        return "formasi"
    elif any(kw in query_lower for kw in faq_keywords):
        return "faq"
    else:
        return "faq"  # default fallback


In [ ]:
# =========================================
# 5️⃣ SETUP LLM DAN PROMPT
# =========================================
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key="xxxxxxxxxx",
    temperature=0.2
)

schemas = [ResponseSchema(name="jawaban", description="Jawaban yang ringkas dan akurat.")]
parser = StructuredOutputParser.from_response_schemas(schemas)
format_instructions = parser.get_format_instructions()


In [13]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template("""
Kamu adalah asisten resmi dari **Badan Kepegawaian Negara (BKN)** yang bertugas membantu masyarakat memahami seluruh hal tentang **Aparatur Sipil Negara (ASN)**, termasuk CPNS, PPPK, formasi, regulasi, serta pertanyaan umum terkait pendaftaran.

## 🎯 TUJUAN
Berikan jawaban **jelas, ringkas, dan akurat** berdasarkan konteks dokumen yang tersedia.  
Gunakan bahasa formal namun mudah dipahami masyarakat umum.

---

## 🧠 INSTRUKSI UTAMA
1. **Identifikasi jenis pertanyaan pengguna:**
   - Jika terkait *formasi jabatan atau kualifikasi pendidikan* → gunakan konteks FORMASI.  
   - Jika pertanyaan bersifat *umum atau konseptual tentang ASN, CPNS, PPPK, SSCASN, NIK, atau regulasi* → gunakan konteks FAQ.
   - Jika tidak ada kecocokan sama sekali → tampilkan pesan default.

2. **Format jawaban berdasarkan jenisnya:**

### a. Jika konteks FORMASI:
   - Tampilkan informasi dengan format:
     ```
     🏢 Instansi: [nama instansi]
     💼 Jabatan: [nama jabatan]
     🎓 Kualifikasi: [kualifikasi pendidikan]
     👥 Kebutuhan: [jumlah kebutuhan]
     💰 Gaji (jika tersedia): [kisaran gaji]
     📘 Dasar regulasi (jika ada): [nama regulasi]
     ```
   - Jika lebih dari satu formasi relevan:
     - Tampilkan SEMUA hasil.
     - Urutkan berdasarkan jumlah kebutuhan (terbanyak → tersedikit).

### b. Jika konteks FAQ:
   - Ambil jawaban yang paling sesuai dengan pertanyaan.
   - Sajikan ringkas tapi tetap lengkap, lalu tambahkan:
     - 📘 Dasar hukum (nama peraturan)
     - 🔗 Sumber rujukan (tautan dokumen resmi)
   - Contoh:
     ```
     ASN (Aparatur Sipil Negara) terdiri dari PNS dan PPPK ...
     📘 Dasar: Undang-Undang Nomor 20 Tahun 2023 tentang ASN
     🔗 https://peraturan.bpk.go.id/Details/269470/uu-no-20-tahun-2023
     ```

3. **Batasan:**
   - Jangan mengarang jawaban yang tidak ada dalam konteks.
   - Jika data tidak ditemukan → tampilkan:
     > "Maaf, saya tidak menemukan informasi tersebut dalam dokumen yang tersedia."

4. **Penutupan Jawaban:**
   - Selalu akhiri dengan:
     > "Untuk informasi lebih lengkap dan terkini, silakan kunjungi https://sscasn.bkn.go.id"

---

## 🔍 INPUT PENGGUNA
Pertanyaan pengguna:
{question}

---

## 📚 KONTEKS DOKUMEN (bisa berisi data formasi & FAQ)
{context}

{format_instructions}
""")


In [14]:
# =========================================
# 6️⃣ FUNGSI QUERY OTOMATIS (FAQ / FORMASI)
# =========================================
def ask_bkn(query: str):
    tipe = detect_query_type(query)
    print(f"🧭 Jenis pertanyaan terdeteksi: {tipe}")

    db = db_formasi if tipe == "formasi" else db_faq
    retriever = db.as_retriever(search_kwargs={"k": 3})

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt_template.partial(format_instructions=format_instructions)}
    )

    response = qa_chain.invoke({"query": query})
    raw_answer = response.get("result", "")
    try:
        parsed = parser.parse(raw_answer)
        return parsed.get("jawaban", raw_answer)
    except Exception:
        return raw_answer



In [15]:

# =========================================
# 7️⃣ UJI COBA
# =========================================
print("\n🧠 Tes Pertanyaan FORMASI:")
print(ask_bkn("formasi dengan penempatan di Jakarta dan gaji tertinggi"))

print("\n📘 Tes Pertanyaan FAQ:")
print(ask_bkn("apa itu CPNS dan PPPK?"))


🧠 Tes Pertanyaan FORMASI:
🧭 Jenis pertanyaan terdeteksi: formasi
Berikut adalah formasi dengan penempatan di Jakarta dan kisaran gaji tertinggi yang ditemukan:

🏢 Instansi: Badan Riset dan Inovasi Nasional
💼 Jabatan: PENELITI AHLI MUDA
🎓 Kualifikasi: S3 semua jurusan; S3 Ilmu Komputer; S3 Teknik Informatika; S3 Sistem Informasi; S3 Statistika; S3 Matematika; S3 Fisika; S3 Kimia; S3 Biologi; S3 Kesehatan Masyarakat; S3 Ilmu Ekonomi; S3 Manajemen; S3 Ilmu Pemerintahan; S3 Sosiologi; S3 Antropologi; S3 Ilmu Lingkungan; S3 Teknik Elektro; S3 Agronomi; S3 Farmasi; S3 Ilmu Pendidikan
👥 Kebutuhan: 75
💰 Gaji (jika tersedia): 7 - 11 juta

🏢 Instansi: Badan Riset dan Inovasi Nasional
💼 Jabatan: PENELITI AHLI MUDA
🎓 Kualifikasi: S3 semua jurusan; S3 Ilmu Komputer; S3 Teknik Informatika; S3 Sistem Informasi; S3 Statistika; S3 Matematika; S3 Fisika; S3 Kimia; S3 Biologi; S3 Kesehatan Masyarakat; S3 Ilmu Ekonomi; S3 Manajemen; S3 Ilmu Pemerintahan; S3 Sosiologi; S3 Antropologi; S3 Ilmu Lingkungan; 